In [ ]:
import osmnx as ox
import geopandas as gpd
import pandas as pd
import pickle
import json
import os

# Data file
file_path = "hungarian_microregions.xlsx"
regions = pd.read_excel(file_path)

# Saving folder
save_dir = "microregions_data"
os.makedirs(save_dir, exist_ok=True)

def download_and_save_osm_data(regions_name):
    try:
        # Boundary geometry
        regions = ox.geocode_to_gdf(regions_name)
        regions.to_file(f"{save_dir}/{regions_name}_boundary.geojson", driver="GeoJSON")
        
        # Roads
        roads = ox.graph_from_place(regions_name, network_type="drive")
        with open(f"{save_dir}/{regions_name}_roads.pkl", "wb") as f:
            pickle.dump(roads, f)

        # Parks
        parks = ox.features_from_place(regions_name, tags={"leisure": "park"})
        parks.to_file(f"{save_dir}/{regions_name}_parks.geojson", driver="GeoJSON")

        # Schools
        schools = ox.features_from_place(regions_name, tags={"amenity": "school"})
        schools.to_file(f"{save_dir}/{regions_name}_schools.geojson", driver="GeoJSON")

        # Museums
        museums = ox.features_from_place(regions_name, tags={"tourism": "museum"})
        museums.to_file(f"{save_dir}/{regions_name}_museums.geojson", driver="GeoJSON")

        print(f"Data downloaded and saved: {regions_name}")
    except Exception as e:
        print(f"Error in {regions_name} process: {e}")

# Loop for all download
for regions_name in regions['Name']:
    download_and_save_osm_data(regions_name)

print("All microregions OSM data downloaded and saved!")